# Part 1

Part 1 asked us to fit a Poisson Regression GLM to the crime data in Alameda county using the weather data. To accomplish this task, we used some of the functions we had written in previous assignments. Additionally, it takes awhile for some of the data to be computed, so we saved some of it in pickle files and reused them when possible. We also display the original code used to get the data. The first thing we needed to do was get the weather data for East Alameda and West Alameda.

In [20]:
def get_alameda_county_points():
    """ Calculates all the grid points (lat, lon) inside Alameda County
    Grid automatically includes Summit Reservoir (37.905098, -122.272225)
    Begin at northwest corner of bounding box and move in increment of 5 miles
    in the south and east directions until we reach the southeast corner,
    recording all grid points that fall inside Alameda County.

    return:
        list: list of (lat, lon) points inside Alameda County
    """
    # bounding box around Alameda County
    north = 38
    west = -122.4
    south = 37.4
    east = -121.4

    # grid automatically includes Summit Reservoir (37.905098, -122.272225)
    grid_points = [(37.905098, -122.272225)]
    curr = [north, west]

    # while current point is within the north / south bounds
    while curr[0] > south:
        # dynamically update lat and lon increment based on curr point
        destE = vincenty(miles=5).destination(curr, 90) # point 5 miles east of curr
        lon_increment = destE.longitude - curr[1]
        destS = vincenty(miles=5).destination(curr, 180) # point 5 miles south of curr
        lat_increment = curr[0] - destS.latitude

        # while current point is within the east / west bounds
        while curr[1] < east:
            if (rg.search(curr)[0]['admin2'] == "Alameda County"):
                grid_points.append(tuple(curr))
            curr[1] += lon_increment
        curr[0] -= lat_increment
        curr[1] = west

    return grid_points


In [21]:
def divide_alameda_points():
    """ Return a list of West Alameda points and a list of East Alameda points

    return:
        list: list of West Alameda (lat, lon) points
        list: list of East Alameda (lat, lon) points
    """
    grid_points = get_alameda_county_points()
    border_west = -122.06 # westernmost point of "East Alameda", points to the left are in West Alameda
    border_east = -121.82 # easternmost point in "West Alameda", points to the right are in East Alameda

    # zipcodes identified to be in West or East Alameda based on Google Maps
    zip_loc = {
        'west': [94539, 94552, 94544, 94536, 94537, 94538],
        'east': [94568, 94566, 94586]
    }

    west_alameda = []
    east_alameda = []
    for point in grid_points:
        if point[1] < border_west:
            west_alameda.append(point)
        elif point[1] > border_east:
            east_alameda.append(point)
        else:
            # classify ambiguous points that lie between border_west and border_east
            try:
                # get zipcode of unknown point
                unknown_zip = int(re.findall(r'\d+', geolocator.reverse(point).raw['address']['postcode'])[0])
                if unknown_zip in zip_loc['west']:
                    west_alameda.append(point)
                elif unknown_zip in zip_loc['east']:
                    east_alameda.append(point)
            except:
                # Some points represent landmarks or roads that stretch multiple zipcodes, 
                # causing the above code to error. We handle these cases separately.
                unknown_loc = geolocator.reverse(point).raw['address']['hamlet']
                if unknown_loc == 'Kilkare Woods':
                    west_alameda.append(point)
            
    return west_alameda, east_alameda

In [22]:
def get_corrected_weather_data(load_pickle=False):
    """ Get corrected weather data for the areas of interest

    return
        DataFrame: df from west Alameda
        DataFrame: df from east Alameda
        DataFrame: df from Alameda
    """
    w_pts, e_pts = divide_alameda_points()

    alameda = pickle.load(open('Assignment6/alameda_weather', 'rb'))
    east_weather = pickle.load(open('Assignment6/east_weather_file', 'rb'))
    west_weather = pickle.load(open('Assignment6/west_weather_file', 'rb'))

    # load from assignment 3 function
    if not load_pickle:
        alameda = main(elements=['TMAX', 'PRCP'])
        east_weather = main(points = e_pts, elements=['TMAX', 'PRCP'])
        west_weather = main(points = w_pts, elements=['TMAX', 'PRCP'])
    return west_weather, east_weather, alameda

We pull the data from the `main` function in assignment 3. It essentially takes a list of points, identifies weather stations within a particular radius of those points, and then it creates a data frame for each year and month combinations that bins the inverse weighted average temperature from the weather stations. The temperature reported is corrected for bias. After we do this, we need to get the crime data.

In [23]:
def get_crime_data(load_pickle=False):
    """ Creates a DataFrame of crime data for Alameda County from 1980 to 2009
        Pickle this dataframe for future use

    return:
        DataFrame: Crime data DF
    """
    if load_pickle:
        return pickle.load(open('Assignment6/df_1980_2009.pkl', 'rb'))
    # get list of dta files for 1980 - 2009 data
    crime_data_path = "Assignment6/data/ucr_offenses_known_monthly_1960_2016_dta/"
    dta_files = [f for f in os.listdir(crime_data_path) if f[-4:] == '.dta']
    dta_files_80_09 = [f for f in dta_files if int(f[-8:-4]) >= 1980 and int(f[-8:-4]) <= 2009]

    # 1980 - 2009 crime data
    df_list = []
    for f in dta_files_80_09:
        df = pd.read_stata(crime_data_path + f)
        # get only Alameda County data point
        df = df[(df['fips_state_code'] == '06') & (df['fips_county_code'] == '001')]
        df_list.append(df)
    df_80_09 = pd.concat(df_list)
    df_80_09.to_pickle("Assignment6/data/df_1980_2009.pkl")
    return df_80_09

We get the crime data by concatenating all the relevant years and then pickling the data. Once again, we pickle it because this operation takes a really long time. The next step for us is to clean the crime data. We needed to put it into a form that can be easily joined with the weather data to create our design matrix for the Poisson model.

In [24]:
def clean_crime_data(alameda_crime):
    """ Take crime data, clean it, and divide it into the areas of interest
    
    args:
        DataFrame: crime data DF
    return:
        DataFrame: df from west Alameda
        DataFrame: df from east Alameda
        DataFrame: df from Alameda
    """
    # get sum of crime incidences
    alameda_crime['total_crime'] = alameda_crime.filter(regex='_total').sum(axis=1)
    east = alameda_crime[alameda_crime['zip_code'].isin(classify_zip['east'])]
    west = alameda_crime[alameda_crime['zip_code'].isin(classify_zip['west'])]
    
    # get cleaned data
    east_cleaned = east.groupby(['year', 'month']).sum()[['total_crime']].reset_index()
    west_cleaned = west.groupby(['year', 'month']).sum()[['total_crime']].reset_index()
    alameda_cleaned = alameda_crime.groupby(['year', 'month']).sum()[['total_crime']].reset_index()

    # change month name to month number
    east_cleaned['month'] = pd.to_datetime(east_cleaned['month'], format='%B').dt.month
    west_cleaned['month'] = pd.to_datetime(west_cleaned['month'], format='%B').dt.month
    alameda_cleaned['month'] = pd.to_datetime(alameda_cleaned['month'], format='%B').dt.month

    return west_cleaned, east_cleaned, alameda_cleaned

This function calculates the total crime in Alameda. Then, it divides the county into East and West. Next, we group the data by month and year combination and then sum it. Additionally, we need to convert the month strings into month indexes so that it is easy for us to merge it with our weather data frame. Our next step is to merge the two and data frames to create our data matrix.

In [25]:
def get_data_matrix(weather, crime):
    """ Gets the regression data matrix from the weather and crime data

    args:
        weather: weather DataFrame
        crime: crime DataFrame
    return:
        ndarray: design matrix
        ndarray: dependent variable
    """
    # merge temperature with precipitation
    a_temp, a_prcp = weather
    a_temp.columns = a_temp.columns.astype(str)
    a_prcp.columns = a_prcp.columns.astype(str)
    merged_weather = a_temp.join(a_prcp)

    # merge crime with weather
    merged = pd.merge(crime, merged_weather.reset_index(), on=['year', 'month'], how='inner').fillna(0.0) 
    merged_data = merged.sort_values(by=['year', 'month']).reset_index(drop=True).drop(['year', 'month'], axis=1)

    # get x and y values
    y = merged_data['total_crime']
    x = merged_data.drop('total_crime', axis=1).values

    return x, y

Then we can take the weather data and crime data and combine it form our X and Y data. We merge the crime data with the weather data and then pull the dependent variable and the covariate values out. Finally, using the data we need to create our model.

In [26]:
def get_model(x, y):
    """ Gets fitted GLM Poisson regression from x and y

    args:
        x: design matrix
        y: dependent variable
    return:
        GLM: model
    """
    return sm.GLM(y, x, family=sm.families.Poisson()).fit()

This simply takes in an X and a Y and returns the fitted Poisson model.

In [27]:
def run_assignment6():
    """ Run assignment 6 to get the models

    return:
        GLM: west
        GLM: east
        GLM: alameda
    """
    west, east, alameda = get_corrected_weather_data(load_pickle=True)
    crime_data = get_crime_data(load_pickle=True)
    west_crime, east_crime, alameda_crime = clean_crime_data(crime_data)

    west_x, west_y = get_data_matrix(west, west_crime)
    east_x, east_y = get_data_matrix(east, east_crime)
    alameda_x, alameda_y = get_data_matrix(alameda, alameda_crime)

    return get_model(west_x, west_y), get_model(east_x, east_y), get_model(alameda_x, alameda_y)
